# Westeros Tutorial - Introducing emissions (part2/2): Adding emission taxes

In the previous tutorial (``westeros_emissions_bounds.ipynb``), we showed how to introduce emissions into a stylized energy systems model, and what happens if you put a constraint on total CO2 emissions.

Now, we will tackle the complementary policy to emissions constraints, namely emissions taxes.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully
- You have run the tutorial on introducing emissions (``westeros_emissions_bounds.ipynb``)

In [ ]:
import ixmp

mp = ixmp.Platform()

In [ ]:
import message_ix

model = "Westeros Electrified"
base = message_ix.Scenario(mp, model=model, scenario="baseline")

## Load the scenario with an emission bound and look at the result in more detail

In [ ]:
scen_bd = message_ix.Scenario(mp, model=model, scenario="emission_bound")

In [ ]:
# Emissions in MtCO2
emissions = scen_bd.var("EMISS", {"node": "Westeros"})
emissions

In [ ]:
# Price in USD/tCO2
emission_prices = scen_bd.var("PRICE_EMISSION")
emission_prices

When setting a cumulative bound, the undiscounted price of emission is the same in different model years (see the marginals of
equation `"EMISSION_EQUIVALENCE"`). However, considering the year-to-year discount factor, we observe an ascending trend in
emission prices shown in `"PRICE_EMISSION"` above. This means the emission price in later years is higher as the value of money in
the future is lower compared to today. 

## Make a new scenario with emission bounds by year

In the previous example, we imposed a bound on emissions over the entire model horizon by using the `type_year` as 'cumulative'
in the parameter `"bound_emission"`. Now, we will create a similar scenario, but the emission constraint will be defined per year.

For the sake of comparison, the per-year emission values will be chosen exactly in line with the optimal emission trajectory
obtained from the solution of the previous scenario.

In [ ]:
scen_bd_by_year = base.clone(
    model, "carbon_bound_by_year", "introducing a carbon tax", keep_solution=False
)
scen_bd_by_year.check_out()

In [ ]:
scen_bd_by_year.add_set("emission", "CO2")
scen_bd_by_year.add_cat("emission", "GHG", "CO2")
scen_bd_by_year.add_par("emission_factor", scen_bd.par("emission_factor"))
scen_bd_by_year.par("emission_factor")

We specify the model horizon and subsequently add an emission bound parameter to the model. Note that the key `value` here takes the values of the `lvl` column of the `emissions` dataframe printed some cells above.

In [ ]:
horizon = [700, 710, 720]

bd_emission = message_ix.make_df(
    "bound_emission",
    node="Westeros",
    type_year=horizon,
    type_tec="all",
    unit="MtCO2",
    type_emission="GHG",
    value=emissions.set_index("year").lvl,
)
scen_bd_by_year.add_par("bound_emission", bd_emission)
scen_bd_by_year.par("bound_emission")

In [ ]:
scen_bd_by_year.commit(comment="emission bound by year")

In [ ]:
scen_bd_by_year.solve()

In [ ]:
emission_prices_by_year = scen_bd_by_year.var("PRICE_EMISSION")
emission_prices_by_year

Comparing the emission prices between the two scenarios at this stage, we see that the values are not identical.
The reason is that when we introduce emission bounds per year, the price of emission in each year reflects the cost occuring
when reducing one more unit of emission in that year.
However, in the scenario with a cumulative bound over the entire model horizon, the price of emission reflects the cost of the
system in reducing one more unit of emission over the entire model horizon.

## Setting an emissions tax instead of a bound

Again, we choose the emissions prices from the first example (with a cumulative bound) as the tax level over time.

In [ ]:
scen_tax = base.clone(
    model, "carbon_tax", "introducing a carbon tax", keep_solution=False
)
scen_tax.check_out()

In [ ]:
scen_tax.add_set("emission", "CO2")
scen_tax.add_cat("emission", "GHG", "CO2")
scen_tax.add_par("emission_factor", scen_bd.par("emission_factor"))

In [ ]:
scen_tax.add_set("type_year", [700, 710, 720])

# In case you need to add the new unit to the platform
mp.add_unit("USD/tCO2")

tax_emission = message_ix.make_df(
    "tax_emission",
    node="Westeros",
    type_year=[700, 710, 720],
    type_tec="all",
    unit="USD/tCO2",
    type_emission="GHG",
    value=emission_prices.set_index("year").lvl,
)
scen_tax.add_par("tax_emission", tax_emission)
scen_tax.par("tax_emission")

In [ ]:
scen_tax.commit(comment="setting taxes on emissions")

In [ ]:
scen_tax.set_as_default()

In [ ]:
scen_tax.solve()

In [ ]:
scen_tax.var("EMISS", {"node": "Westeros"})

# Exercises
- How do these prices compare to the scenario with a cumulative emission bound (`scen_bd`)?
- Try setting the emission tax again by using emission prices obtained from the scenario with yearly bounds on emissions (`scen_bd_by_year`). What is the difference in emissions (i.e., variable `EMISS`)?

## Close the connection to the database

In [ ]:
mp.close_db()